In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
Renewable_Energy = "Solar_PBE"

try:
    address = "/content/drive/My Drive/Doctor/Research/DRL/Colab/"
    data_train_csv1 = pd.read_csv(address+Renewable_Energy+'_16.csv', index_col=0)

except:
    address = ""
    data_train_csv1 = pd.read_csv(address+Renewable_Energy+'_16.csv', index_col=0)

data_train_csv2 = pd.read_csv(address+Renewable_Energy+'_17.csv', index_col=0)
data_train_csv  = pd.concat([data_train_csv1, data_train_csv2])
data_val_csv    = pd.read_csv(address+Renewable_Energy+'_18.csv', index_col=0)
data_test_csv   = pd.read_csv(address+Renewable_Energy+'_19.csv', index_col=0)

In [ ]:
# Data Preprocessing

unit = 4 #unit: 15 minute

RE_Capacity1 = max(data_train_csv['Power(MW)'])
RE_Capacity2 = max(data_val_csv['Power(MW)'])
RE_Capacity3 = max(data_test_csv['Power(MW)'])

size_train0 = int(len(data_train_csv)/unit)
size_val0   = int(len(data_val_csv)/unit)
size_test0  = int(len(data_test_csv)/unit)

data_train0 = []
data_train  = []
for i in range(size_train0):
    data_train0 += [round(pd.Series.mean(data_train_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity1, 3)]
    data_train  += [data_train0[i]] if data_train0[i] > 0 else []

data_val0 = []
data_val  = []
for i in range(size_val0):
    data_val0 += [round(pd.Series.mean(data_val_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity2, 3)]
    data_val  += [data_val0[i]] if data_val0[i] > 0 else []

data_test0 = []
data_test  = []
for i in range(size_test0):
    data_test0 += [round(pd.Series.mean(data_test_csv['Power(MW)'][i*unit:(i+1)*unit])/RE_Capacity3, 3)]
    data_test  += [data_test0[i]] if data_test0[i] > 0 else []

In [ ]:
# LSTM

n_layers       = 1
in_size        = 1
hidden_size    = 16
out_size       = 1
batch_size     = 128
learning_rate  = 0.001

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.fc_in  = nn.Linear(in_size, hidden_size)
        self.rnn    = nn.LSTM(hidden_size, hidden_size, n_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_size, out_size)
    
    def forward(self, x, hidden):
        x = F.relu(self.fc_in(x))
        x = x.view(1, -1, hidden_size)
        x, hidden = self.rnn(x, hidden)
        out = self.fc_out(x)
        out = F.relu(out.view(-1, out_size))
        return out, hidden
        
def train_net(model, batch, optimizer):
    x, h, y = batch[0], batch[1], batch[2]
    loss = F.mse_loss(model.forward(x, h)[0], y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
# Training LSTM

total_epoch    = 100
print_interval = 10

model = LSTM()
size_train = len(data_train)
size_val = len(data_val)
size_test = len(data_test)

train_input = np.zeros((size_train-1, 1))
train_output = np.zeros((size_train-1, 1))
for i in range(size_train-1):
    train_input[i,:] = data_train[i]
    train_output[i,:] = data_train[i+1]

val_input = np.zeros((size_val-1, 1))
val_output = np.zeros((size_val-1, 1))
for i in range(size_val-1):
    val_input[i,:] = data_val[i]
    val_output[i,:] = data_val[i+1]

test_input = np.zeros((size_test-1, 1))
test_output = np.zeros((size_test-1, 1))
for i in range(size_test-1):
    test_input[i,:] = data_test[i]
    test_output[i,:] = data_test[i+1]

total_batch = int((size_train-1)/batch_size) + 1
pred_train, pred_val, pred_test = [], [], [] # Predicted Value
mape_train, mape_val, mape_test = [], [], [] # Mean Absolute Percentage Error

hidden = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_x = torch.tensor(train_input[batch_size*i:batch_size*(i+1),:] ,dtype=torch.float)
        batch_y = torch.tensor(train_output[batch_size*i:batch_size*(i+1),:] ,dtype=torch.float)
        batch = [batch_x, hidden, batch_y]
        train_net(model, batch, optimizer)
        _, hidden = model.forward(batch_x, hidden)
        hidden = (hidden[0].detach(), hidden[1].detach())

    hidden = (torch.zeros([n_layers, 1, hidden_size], dtype=torch.float), torch.zeros([n_layers, 1, hidden_size], dtype=torch.float))
    if epoch == 0 or (epoch+1) % print_interval == 0:
        train_predict = model.forward(torch.tensor(train_input, dtype=torch.float), hidden)[0].detach().numpy()
        pred_train += [list(train_predict.flatten())]
        mape_train += [list(np.abs(train_predict - train_output).flatten()/train_output.flatten())]
        
        val_predict = model.forward(torch.tensor(val_input, dtype=torch.float), hidden)[0].detach().numpy()
        pred_val += [list(val_predict.flatten())]
        mape_val += [list(np.abs(val_predict - val_output).flatten()/val_output.flatten())]
        
        test_predict = model.forward(torch.tensor(test_input, dtype=torch.float), hidden)[0].detach().numpy()
        pred_test += [list(test_predict.flatten())]
        mape_test += [list(np.abs(test_predict - test_output).flatten()/test_output.flatten())]

        MAPE_train = round(100*np.mean(mape_train[-1]),2)
        MAPE_val   = round(100*np.mean(mape_val[-1]),2)
        MAPE_test  = round(100*np.mean(mape_test[-1]),2)

        print("epoch: {}".format(epoch+1))
        print("MAPE_train: {}%".format(MAPE_train).ljust(25), end="")
        print("MAPE_val: {}%".format(MAPE_val).ljust(25), end="")
        print("MAPE_test: {}%".format(MAPE_test).ljust(25))
        print("------------------------------------------------------------------------------------------")

epoch: 1
MAPE_train: 1194.55%     MAPE_val: 1197.87%       MAPE_test: 1170.64%      
------------------------------------------------------------------------------------------
epoch: 10
MAPE_train: 95.96%       MAPE_val: 82.77%         MAPE_test: 89.19%        
------------------------------------------------------------------------------------------
epoch: 20
MAPE_train: 63.18%       MAPE_val: 56.31%         MAPE_test: 58.71%        
------------------------------------------------------------------------------------------
epoch: 30
MAPE_train: 51.79%       MAPE_val: 47.74%         MAPE_test: 48.8%         
------------------------------------------------------------------------------------------
epoch: 40
MAPE_train: 44.19%       MAPE_val: 41.6%          MAPE_test: 42.33%        
------------------------------------------------------------------------------------------
epoch: 50
MAPE_train: 40.88%       MAPE_val: 39.53%         MAPE_test: 39.31%        
------------------------------

In [ ]:
# Produce results

# select_num = np.argmin(np.mean(mape_val,axis=1))
# select_train = pd.DataFrame(np.array(pred_train[select_num][:]))
# select_val = pd.DataFrame(np.array(pred_val[select_num][:]))
# select_test = pd.DataFrame(np.array(pred_test[select_num][:]))
# select_train.to_csv(address+Renewable_Energy+"_Model1_train.csv")
# select_val.to_csv(address+Renewable_Energy+"_Model1_val.csv")
# select_test.to_csv(address+Renewable_Energy+"_Model1_NEC.csv")